In [8]:
import requests
import json
import os
from dotenv import load_dotenv
from geopy.geocoders import Nominatim

In [9]:
import google.generativeai as genai
from google.generativeai import GenerationConfig

In [12]:
load_dotenv()

True

In [14]:
def get_coordinates(city_name):
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city_name)
    if location:
        return (location.latitude, location.longitude)
    return None

In [16]:
genai.configure(api_key=os.getenv("GEMINI_KEY"))

In [18]:
base_url="http://api.weatherapi.com/v1"

In [20]:
gc=GenerationConfig(
    temperature=0.5,
    max_output_tokens=35
)
model=genai.GenerativeModel("gemini-2.0-flash",generation_config=gc)

In [22]:
chat=genai.ChatSession(model=model,history=[])

In [24]:
chat.send_message("""
                    You are given as input a city or coordinates for which the user is to find out the weather...
                    RETURN THE NAME OF THE CITY OR REGION AS IT WOULD BE INDEXED IN GEOPY  
                  """)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Okay, I understand. My task is to extract the city or region name from the user's input in a format suitable for use with the GeoPy library. This means"
              }
            ],
            "role": "model"
          },
          "finish_reason": "MAX_TOKENS",
          "avg_logprobs": -0.13160305023193358
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 45,
        "candidates_token_count": 35,
        "total_token_count": 80
      }
    }),
)

In [25]:
fc=int(input("Do you wish to check for the forecast, or the current weather? Press 1 for forecast and 2 for current weather...:\n"))
fc

Do you wish to check for the forecast, or the current weather? Press 1 for forecast and 2 for current weather...:
 2


2

In [28]:
c="/forecast.json" if fc==1 else "/current.json"
alerts=input("Do you wish to see weather alerts? Yes or No?:\n")
alerts

Do you wish to see weather alerts? Yes or No?:
 y


'y'

In [30]:
days = int(input("Enter the number of days for the forecast...")) if fc==1 else None
days

In [32]:
aqi=input("Do you wish to see the aqi? Yes or no?\n")
aqi

Do you wish to see the aqi? Yes or no?
 y


'y'

In [34]:
def antipode(coords):
    lat, lon = coords
    antipode_lat = -lat
    antipode_lon = lon + 180 if lon < 0 else lon - 180
    return (antipode_lat, antipode_lon)


In [187]:
query=input("Enter the city or coordinates you wish to check:\n") 
query

Enter the city or coordinates you wish to check:
 New Delhi


'New Delhi'

In [169]:
query=chat.send_message(query).text
query=get_coordinates(query)
query

(34.0536909, -118.242766)

In [147]:
query=antipode(query)
query

(-40.7127281, 105.9939848)

In [189]:
import pandas as pd
from geopy.distance import geodesic
ports = pd.read_csv('WPI_FILE.csv') 
def nearest_port(coords):
    lat,lon=coords[0],coords[1]
    def dist(row):
        return geodesic((lat, lon), (row['LATITUDE'], row['LONGITUDE'])).kilometers
    ports['Distance'] = ports.apply(dist, axis=1)
    return (ports.sort_values('Distance').loc[0,["LATITUDE"]].values.tolist()[0],ports.sort_values('Distance').loc[0,["LONGITUDE"]].values.tolist()[0])



In [191]:
query

'New Delhi'

In [193]:
payload={}
if fc==2:
    payload.update({"key":os.getenv("WEATHER_API"),"q":query,"alerts":"yes" if alerts.lower()=="yes"or alerts.lower()=="y" else "no",
                    "aqi":"yes" if aqi.lower()=="yes"or aqi.lower()=="y" else "no"})
elif fc==1:
    payload.update({"key":os.getenv("WEATHER_API"),"q":query, "days":days,"alerts":"yes" if alerts.lower()=="yes"or alerts.lower()=="y" else "no",
                    "aqi":"yes" if aqi.lower()=="yes"or aqi.lower()=="y" else "no"})


In [195]:
r=requests.get(base_url+c,params=payload)

In [197]:
r.url

'http://api.weatherapi.com/v1/current.json?key=2d6e3ca4ca984830a06131621250404&q=New+Delhi&alerts=yes&aqi=yes'

In [211]:
data=json.loads(r.text)

In [213]:
data

{'location': {'name': 'New Delhi',
  'region': 'Delhi',
  'country': 'India',
  'lat': 28.6,
  'lon': 77.2,
  'tz_id': 'Asia/Kolkata',
  'localtime_epoch': 1744008090,
  'localtime': '2025-04-07 12:11'},
 'current': {'last_updated_epoch': 1744007400,
  'last_updated': '2025-04-07 12:00',
  'temp_c': 36.1,
  'temp_f': 97.0,
  'is_day': 1,
  'condition': {'text': 'Mist',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png',
   'code': 1030},
  'wind_mph': 6.0,
  'wind_kph': 9.7,
  'wind_degree': 122,
  'wind_dir': 'ESE',
  'pressure_mb': 1011.0,
  'pressure_in': 29.85,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 19,
  'cloud': 0,
  'feelslike_c': 34.5,
  'feelslike_f': 94.2,
  'windchill_c': 38.9,
  'windchill_f': 101.9,
  'heatindex_c': 38.3,
  'heatindex_f': 100.9,
  'dewpoint_c': -3.6,
  'dewpoint_f': 25.5,
  'vis_km': 5.0,
  'vis_miles': 3.0,
  'uv': 9.0,
  'gust_mph': 7.0,
  'gust_kph': 11.2,
  'air_quality': {'co': 2327.3,
   'no2': 81.77,
   'o3': 70.0,
   'so2': 

In [203]:
current=data["current"]
current

{'last_updated_epoch': 1744007400,
 'last_updated': '2025-04-07 12:00',
 'temp_c': 36.1,
 'temp_f': 97.0,
 'is_day': 1,
 'condition': {'text': 'Mist',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png',
  'code': 1030},
 'wind_mph': 6.0,
 'wind_kph': 9.7,
 'wind_degree': 122,
 'wind_dir': 'ESE',
 'pressure_mb': 1011.0,
 'pressure_in': 29.85,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 19,
 'cloud': 0,
 'feelslike_c': 34.5,
 'feelslike_f': 94.2,
 'windchill_c': 38.9,
 'windchill_f': 101.9,
 'heatindex_c': 38.3,
 'heatindex_f': 100.9,
 'dewpoint_c': -3.6,
 'dewpoint_f': 25.5,
 'vis_km': 5.0,
 'vis_miles': 3.0,
 'uv': 9.0,
 'gust_mph': 7.0,
 'gust_kph': 11.2,
 'air_quality': {'co': 2327.3,
  'no2': 81.77,
  'o3': 70.0,
  'so2': 60.68,
  'pm2_5': 143.375,
  'pm10': 465.275,
  'us-epa-index': 4,
  'gb-defra-index': 10}}

In [205]:
response =  {"Location":data["location"],"Temperature in Celsius":current.get("temp_c"), 
              "Windspeed and Direction": f"""{current.get("wind_kph")} {current.get("wind_dir")}""",
              "Precipitation": f"""{current.get("precip_in")} inches or {current.get("precip_mm")} mm""",
              "Humidity":current.get("humidity"),
              "Cloud Cover": current.get("cloud"),
              "Feels Like": current.get("feelslike_c")} if payload["aqi"]=="no" else {"Location":data["location"],"Temperature in Celsius":current.get("temp_c"), 
              "Windspeed and Direction": f"""{current.get("wind_kph")} {current.get("wind_dir")}""",
              "Precipitation": f"""{current.get("precip_in")} inches or {current.get("precip_mm")} mm""",
              "Humidity":current.get("humidity"),
              "Cloud Cover": current.get("cloud"),
              "Feels Like": current.get("feelslike_c"),
              "AQI": current.get("air_quality")}

In [207]:
for keys,values in response.items():
    if keys!="AQI":
        print(f"{keys} : {values}")
    if keys=="AQI":
        print("AQI:")
        for keys, values in response.get("AQI").items():
            print(f"\t{keys} : {values}")


Location : {'name': 'New Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6, 'lon': 77.2, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1744008090, 'localtime': '2025-04-07 12:11'}
Temperature in Celsius : 36.1
Windspeed and Direction : 9.7 ESE
Precipitation : 0.0 inches or 0.0 mm
Humidity : 19
Cloud Cover : 0
Feels Like : 34.5
AQI:
	co : 2327.3
	no2 : 81.77
	o3 : 70.0
	so2 : 60.68
	pm2_5 : 143.375
	pm10 : 465.275
	us-epa-index : 4
	gb-defra-index : 10
